![](http://osloyi5le.bkt.clouddn.com/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E6%A1%88%E4%BE%8Bbanner%20v2.jpg)

# 稀牛Lab入门操作
如果你已经很熟悉Jupyter notebook了，那就跳过此部分去看代码吧~

--------
### 界面简介

![](http://onm3nv9z1.bkt.clouddn.com/%E6%93%8D%E4%BD%9C%E7%95%8C%E9%9D%A2.png)


--------
### 熟悉工具栏

下图中标识了工具栏区域的主要操作。

![](http://onm3nv9z1.bkt.clouddn.com/%E5%B7%A5%E5%85%B7%E6%A0%8F.png)

-------
### 熟悉代码区操作
![](http://onm3nv9z1.bkt.clouddn.com/%E4%BB%A3%E7%A0%81%E8%A1%8C%E8%BF%90%E8%A1%8C.png)

点击**编辑栏-Cell**，可以选择相应功能，运行部分/全部代码行。

点击**工具栏-运行按钮**可以运行当前行的代码(也可以使用组合快捷键** Shift+Enter **来代替运行按钮)

-------
### 熟悉快捷键操作
其实，除了点击上面的工具栏,大部分操作都可以通过**键盘快捷键**来完成, 可以大大提高操作效率。<br><br>
详细的键盘快捷操作信息可以点击下图这个按钮来查看到。
接下来介绍一些最常用的快捷键操作。<br><br>

![](http://onm3nv9z1.bkt.clouddn.com/%E5%BF%AB%E6%8D%B7%E9%94%AE%E9%94%AE%E7%9B%98.png)
<br><br>
  - **b**：在当前行下面插入新的cell*（命令模式）*
  - **a**：在当前行上面插入新的cell*（命令模式）*
  - **dd **(敲击d键两下)：删除当前cell*（命令模式）*
  - **z**：撤销对某个cell的删除*（命令模式）*
  - **m** : 将当前行由代码模式转换成 Markdown 模式  *(命令模式)*
  - **1**：将当前的cell转化为具有一级标题的 Maskdown *(命令模式)*
  - **2**：将当前的cell转化为具有二级标题的 Maskdown *(命令模式)*
  - **Enter**：当前的cell离开命令模式，进入编辑模式
  - **Esc**：退出当前cell的编辑模式，进入命名模式
  - **Shift+Enter**：执行当前cell，并自动跳到下一个cell
  - **Ctrl+Enter**：执行当前cell，执行后***不自动调转* **到下一个cell
  

#### 案例正式开始~

# 神经网络完成数据非线性切分

notebook作者：[@寒小阳](blog.csdn.net/han_xiaoyang)

神经网络有着非常强的非线性表达能力，可以对复杂的问题进行学习和表达。<br><br>
以分类问题为例，现实生活中的很多分类场景是不可线性切分的，那特征的处理就尤为重要了，有意思的是，手造特征这件事情靠人工来做是非常容易达到瓶颈的，而且每换一个场景都需要重新结合场景构建一次特征，而深度学习这种端到端的学习方式，非常擅长进行表示学习，在每一次的前向计算过程中，就在自动地做特征映射，而到达最后分类层(softmax层)的时候，其实构造出来的特征空间里，样本已经是能近似线性切分的了，于是可以非常好地完成样本的非线性切分。<br><br>
下面我们来看一个例子，用最简单的多层感知器(单隐层)对样本点进行非线性切分。<br><br>
这里为了给大家展示神经网络的训练细节，我们手写了神经网络的前向计算和反向传播，以及梯度下降优化调参。大家在后续的课程里会学到使用AI生态工具库(tensorflow、pytorch、keras)构建神经网络更快捷地解决这个问题。<br><br>
课程链接：https://mooc.study.163.com/smartSpec/detail/1001473001.htm?share=1&shareId=1015252963

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
# 手动生成一个随机的平面点分布，并画出来
np.random.seed(0)
X, y = make_moons(200, noise=0.20)
plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)
plt.show()

画出了这样一幅图
![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/nn-from-scratch-dataset.png)

### 咱们定义一个函数用来画我们分类结果的决策边界（也就是分界线）

In [ ]:
# 咱们先定义一个函数来画决策边界
def plot_decision_boundary(pred_func):

    # 设定最大最小值，附加一点点边缘填充
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # 用预测函数预测一下
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # 然后画出图
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

### 我们先用传统的逻辑回归来做一下分类，并画出判定边界

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
#咱们先来瞄一眼逻辑斯特回归对于它的分类效果
clf = LogisticRegressionCV()
clf.fit(X, y)

# 画一下决策边界
plot_decision_boundary(lambda x: clf.predict(x))
plt.title("Logistic Regression")
plt.show()

咱们看到下面这样的结果<br>
很好理解对吧，因为线性切分，决策边界是直线，这条直线只能尽量让某一类点落在某一侧，但是永远做不到真的完全切分
![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/nn-from-scratch-lr-decision-boundary.png)

### 咱们来试一个简单的人工神经网络
![](http://www.wildml.com/wp-content/uploads/2015/09/nn-from-scratch-3-layer-network.png)
![](http://s0.wp.com/latex.php?zoom=2&latex=%5Cbegin%7Baligned%7D++z_1+%26+%3D+xW_1+%2B+b_1+%5C%5C++a_1+%26+%3D+%5Ctanh%28z_1%29+%5C%5C++z_2+%26+%3D+a_1W_2+%2B+b_2+%5C%5C++a_2+%26+%3D+%5Chat%7By%7D+%3D+%5Cmathrm%7Bsoftmax%7D%28z_2%29++%5Cend%7Baligned%7D&bg=ffffff&fg=000&s=0)
![](http://s0.wp.com/latex.php?zoom=2&latex=W_1+%5Cin+%5Cmathbb%7BR%7D%5E%7B2%5Ctimes500%7D&bg=ffffff&fg=000&s=0)
![](http://s0.wp.com/latex.php?zoom=2&latex=b_1+%5Cin+%5Cmathbb%7BR%7D%5E%7B500%7D&bg=ffffff&fg=000&s=0)
![](http://s0.wp.com/latex.php?zoom=2&latex=W_2+%5Cin+%5Cmathbb%7BR%7D%5E%7B500%5Ctimes2%7D&bg=ffffff&fg=000&s=0)
![](http://s0.wp.com/latex.php?zoom=2&latex=b_2+%5Cin+%5Cmathbb%7BR%7D%5E%7B2%7D&bg=ffffff&fg=000&s=0)

### 这是一个分类问题，因此我们用softmax分类器，并且用交叉熵损失作为损失函数
![](http://s0.wp.com/latex.php?zoom=2&latex=%5Cbegin%7Baligned%7D++L%28y%2C%5Chat%7By%7D%29+%3D+-+%5Cfrac%7B1%7D%7BN%7D+%5Csum_%7Bn+%5Cin+N%7D+%5Csum_%7Bi+%5Cin+C%7D+y_%7Bn%2Ci%7D+%5Clog%5Chat%7By%7D_%7Bn%2Ci%7D++%5Cend%7Baligned%7D++&bg=ffffff&fg=000&s=0)

### BP计算梯度以便使用(随机)梯度下降
![](http://s0.wp.com/latex.php?zoom=2&latex=%5Cbegin%7Baligned%7D++%26+%5Cdelta_3+%3D+%5Chat%7By%7D+-+y+%5C%5C++%26+%5Cdelta_2+%3D+%281+-+%5Ctanh%5E2+z_1%29+%5Ccirc+%5Cdelta_3W_2%5ET+%5C%5C++%26+%5Cfrac%7B%5Cpartial%7BL%7D%7D%7B%5Cpartial%7BW_2%7D%7D+%3D+a_1%5ET+%5Cdelta_3+%5C%5C++%26+%5Cfrac%7B%5Cpartial%7BL%7D%7D%7B%5Cpartial%7Bb_2%7D%7D+%3D+%5Cdelta_3%5C%5C++%26+%5Cfrac%7B%5Cpartial%7BL%7D%7D%7B%5Cpartial%7BW_1%7D%7D+%3D+x%5ET+%5Cdelta2%5C%5C++%26+%5Cfrac%7B%5Cpartial%7BL%7D%7D%7B%5Cpartial%7Bb_1%7D%7D+%3D+%5Cdelta2+%5C%5C++%5Cend%7Baligned%7D++&bg=ffffff&fg=000&s=0)

In [ ]:
num_examples = len(X) # 样本数
nn_input_dim = 2 # 输入的维度
nn_output_dim = 2 # 输出的类别个数
 
# 梯度下降参数
epsilon = 0.01 # 学习率
reg_lambda = 0.01 # 正则化参数

# 定义损失函数
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # 向前推进，前向运算
    z1 = X.dot(W1) + b1 
    a1 = np.tanh(z1)    
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # 计算损失
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    # 也得加一下正则化项
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

In [ ]:
# 完整的训练建模函数定义
def build_model(nn_hdim, num_passes=20000, print_loss=False):
    '''
    参数：
    1) nn_hdim: 隐层节点个数
    2）num_passes: 梯度下降迭代次数
    3）print_loss: 设定为True的话，每1000次迭代输出一次loss的当前值
    '''
    # 随机初始化一下权重呗
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))
 
    # 这是咱们最后学到的模型
    model = {}
     
    # 开始梯度下降...
    for i in range(0, num_passes):
 
        # 前向运算计算loss
    
    #####   Code here！   #####
        # 3行代码写出Z2的定义
      
    
    
    
    
    
    
        
    ####    Code end.     ####
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
 
        # 反向传播
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)
 
        # 加上正则化项
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1
 
        # 梯度下降更新参数
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
    #####   Code here！   #####
        # 2行代码分别写出w2和b2的定义（别忘了现在在循环里哟~）

        
        
        
        
        
    ####    Code end.     ####
    
        # 得到的模型实际上就是这些权重
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
         
        # 如果设定print_loss了，那我们汇报一下中间状况
        if print_loss and i % 1000 == 0:
          print("在迭代%i轮后的损失函数值为: %f" %(i, calculate_loss(model)))
     
    return model

In [ ]:
# 判定结果的函数
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # 前向运算
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    # 计算概率输出最大概率对应的类别
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

In [ ]:
# 建立隐层有3个节点(神经元)的神经网络
model = build_model(3, print_loss=True)
 
# 然后再把决策/判定边界画出来
plot_decision_boundary(lambda x: predict(model, x))
plt.title("Decision Boundary for hidden layer size 3")
plt.show()

### 咱们看到这样一个图，隐层有3个节点的神经网络能够区分得还不错
![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/nn-from-scratch-h3.png)

In [ ]:
# 然后听闻你想知道不同的隐层神经元个数对结果的影响？
# 那咱们来一起看看吧
plt.figure(figsize=(16, 32))
# 设定不同的隐层节点(神经元)个数
hidden_layer_dimensions = [1, 2, 3, 4, 5, 20, 50]
for i, nn_hdim in enumerate(hidden_layer_dimensions):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer size %d' % nn_hdim)
    model = build_model(nn_hdim)
    plot_decision_boundary(lambda x: predict(model, x))
plt.show()

### 结果差不多是这个样子的
![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/nn-from-scratch-hidden-layer-varying.png)

## 思考题
传说中的“炼丹”要不要来体验一下~

尝试修改一下下列4个参数吧：
<br><br>
epsilon        # 学习率

reg_lambda     # 正则化参数

nn_hdim        # 隐层节点个数

num_passes     # 梯度下降迭代次数

看看有什么不一样~
<br><br>


你的心得体会要不要跟大家分享下~欢迎加入体验课讨论区【QQ：892843111】,这里还有上面代码填空部分的答案哟~<br><br>
课程链接：https://mooc.study.163.com/smartSpec/detail/1001473001.htm?share=1&shareId=1015252963

版权归 © 稀牛学院 所有 保留所有权利